In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
import re

# Load Excel workbook
file_path = r'C:\Users\user\Desktop\Data_Analyst\TDN_Network\TDN_Alpha_and_Access\AI_Showcase_Virtual_Conf_Full_attendee_list.xlsx'
excel_file = pd.ExcelFile(file_path)
all_sheets = pd.read_excel(file_path, sheet_name=None)

# Extract relevant sheets
df_attendees = all_sheets['Full list of attendees']
df_companies = all_sheets['Companies seeking AI automation']
df_freelancers = all_sheets['AI freelancers & agencies for a']
df_investors = all_sheets['AI investors']
df_startups = all_sheets['AI startups']


In [2]:
# Add Source Column to each DataFrame
df_attendees['Source'] = 'Attendees'
df_companies['Source'] = 'Companies'
df_freelancers['Source'] = 'Freelancers'
df_investors['Source'] = 'Investors'
df_startups['Source'] = 'Startups'

# Identify all possible columns
all_columns = set(df_attendees.columns) | set(df_companies.columns) | set(df_freelancers.columns) | set(df_investors.columns) | set(df_startups.columns)

# Align columns across all DataFrames (add missing columns with NaN)
def align_columns(df, all_columns):
    missing_cols = all_columns - set(df.columns)
    for col in missing_cols:
        df[col] = pd.NA
    return df[list(all_columns)]  # Ensure same column order

df_attendees = align_columns(df_attendees, all_columns)
df_companies = align_columns(df_companies, all_columns)
df_freelancers = align_columns(df_freelancers, all_columns)
df_investors = align_columns(df_investors, all_columns)
df_startups = align_columns(df_startups, all_columns)

# Combine all into one DataFrame
combined_df = pd.concat([df_attendees, df_companies, df_freelancers, df_investors, df_startups], ignore_index=True)

# Final Combined Dataset
print(f"Combined Dataset Shape: {combined_df.shape}")


C:\Users\user\AppData\Local\Temp\ipykernel_2440\1067880734.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df_attendees, df_companies, df_freelancers, df_investors, df_startups], ignore_index=True)


Combined Dataset Shape: (3856, 96)


In [3]:
combined_df.to_excel('Cleaned_Data/Master_Combined_List.xlsx', index=False)


In [4]:
# --- Step 1: Standardize & Rename Important Columns ---
df = combined_df 

# Clean column names globally
df.columns = (
    df.columns.str.strip()
               .str.replace(r'\n', ' ', regex=True)  # Remove line breaks
               .str.replace(r'\s+\(\d+\)', '', regex=True)  # Remove counts like (655)
               .str.replace(r'\s+', ' ', regex=True)  # Normalize multiple spaces to single
)

In [5]:
def smart_rename(col_name):
    # --- Priority 1: Exact Matches ---
    if 'Name' in col_name and 'Company' not in col_name:
        return 'Name'
    elif 'LinkedIn' in col_name:
        return 'LinkedIn'
    elif 'Company Name' in col_name:
        return 'Company'
    elif 'Website' in col_name:
        return 'Website'

    # --- Priority 2: Needs Classification ---
    elif 'Interested in AI automation services' in col_name:
        return 'Wants_Automation'
    elif 'Interested in educating their team' in col_name:
        return 'Wants_Team_Training'
    elif 'Interested in taking an AI automation course' in col_name:
        return 'Wants_Personal_Course'

    # --- Priority 3: Investment Stage ---
    elif 'Pre-seed' in col_name:
        return 'Invest_PreSeed'
    elif 'Seed Startups' in col_name:
        return 'Invest_Seed'
    elif 'Series A' in col_name:
        return 'Invest_SeriesA'
    elif 'Series B' in col_name:
        return 'Invest_SeriesB'
    elif 'Series C+' in col_name:
        return 'Invest_SeriesC'

    # --- Priority 4: Regions ---
    elif 'US' in col_name:
        return 'Region_US'
    elif 'Europe' in col_name:
        return 'Region_Europe'
    elif 'Africa' in col_name:
        return 'Region_Africa'
    elif 'Asia Pacific' in col_name:
        return 'Region_AsiaPacific'
    elif 'Middle East' in col_name:
        return 'Region_MiddleEast'
    elif 'Latin America' in col_name:
        return 'Region_LatinAmerica'
    elif 'Israel' in col_name:
        return 'Region_Israel'
    elif 'Canada' in col_name:
        return 'Region_Canada'
    elif 'UK' in col_name:
        return 'Region_UK'

    # --- Default: Return Original ---
    return col_name

# Apply Smart Renaming to Columns
df = df.rename(columns=lambda x: smart_rename(x))


In [6]:
print(df.columns.tolist())


['Globally – everywhere', 'Seed', 'Region_Europe', 'Website', 'Invest_PreSeed', 'LinkedIn', 'Region_US', 'Region_Africa', 'Region_LatinAmerica', 'WhatsApp Number', 'Company', 'Wants_Personal_Course', 'Region_US', 'Wants_Personal_Course', 'Region_Israel', 'Region_AsiaPacific', 'Region_Africa', 'Invest_SeriesB', 'Region_AsiaPacific', 'Region_MiddleEast', 'Region_Europe', 'Wants_Automation', 'AI freelancers & agencies', 'Raising Globally', 'Bootstrapped', 'Region_Israel', 'Wants_Personal_Course', 'Globally – everywhere', 'Company Headcount', 'Region_Canada', 'Region_Israel', 'Invest_SeriesA', 'Invest_Seed', 'Invest_SeriesC', 'Business owner / CEO / Director', 'Region_Canada', 'Region_Israel', 'Invest_PreSeed', 'AI tools used to automate client processes', 'Region_Europe', 'Region_Africa', 'Invest_SeriesB', 'Region_AsiaPacific', 'General attendees', 'AI startups', 'Wants_Team_Training', 'Wants_Team_Training', 'Region_Canada', 'Region_Africa', 'Wants_Automation', 'Invest_SeriesA', 'Seed Sta

In [ ]:
# --- Step 1: Clean Boolean Flags Robustly ---
def clean_flag(x):
    """
    Converts various formats into True/False.
    Handles scalar, list, Series, and malformed entries gracefully.
    """
    try:
        if isinstance(x, (list, pd.Series)):
            x = next((i for i in x if pd.notnull(i)), None)

        if pd.isna(x):
            return False
        if isinstance(x, str):
            x_clean = x.strip().lower()
            return x_clean in ['1', 'true', 'yes']
        return bool(x)
    except Exception as e:
        print(f"[ERROR] Cleaning Flag Value: {x} — {e}")
        return False

# --- Step 2: Define Column Groups ---
core_columns = ['Name', 'LinkedIn', 'Company', 'Website']
need_columns = ['Wants_Automation', 'Wants_Team_Training', 'Wants_Personal_Course']
invest_columns = ['Invest_PreSeed', 'Invest_Seed', 'Invest_SeriesA', 'Invest_SeriesB', 'Invest_SeriesC']
region_columns = ['Region_US', 'Region_Europe', 'Region_Africa', 'Region_AsiaPacific', 
                  'Region_MiddleEast', 'Region_LatinAmerica', 'Region_Israel', 
                  'Region_Canada', 'Region_UK']

# --- Step 3: Clean Flags Across All Groups ---
for col in need_columns + invest_columns + region_columns:
    if col in df.columns:
        df[col] = df[col].apply(clean_flag)
    else:
        print(f"[WARNING] Column '{col}' not found in DataFrame.")

# --- Step 4: Filter Data to Relevant Columns ---
available_columns = [col for col in core_columns + need_columns + invest_columns + region_columns if col in df.columns]
cleaned_df = df[available_columns].copy()

# --- Step 5: Summarize AI Needs, Investment Interests & Regions ---
def summarize_needs(row):
    needs = []
    if row.get('Wants_Automation'): needs.append('Automation Service')
    if row.get('Wants_Team_Training'): needs.append('Team Training')
    if row.get('Wants_Personal_Course'): needs.append('Personal Course')
    return ', '.join(needs) if needs else 'No AI Need'

def summarize_investments(row):
    levels = []
    if row.get('Invest_PreSeed'): levels.append('Pre-Seed')
    if row.get('Invest_Seed'): levels.append('Seed')
    if row.get('Invest_SeriesA'): levels.append('Series A')
    if row.get('Invest_SeriesB'): levels.append('Series B')
    if row.get('Invest_SeriesC'): levels.append('Series C+')
    return ', '.join(levels) if levels else 'No Investment Interest'

def summarize_regions(row):
    regions = []
    for region_col in region_columns:
        if row.get(region_col):
            regions.append(region_col.replace('Region_', '').replace('_', ' '))
    return ', '.join(regions) if regions else 'No Regional Focus'




C:\Users\user\AppData\Local\Temp\ipykernel_2440\2436564540.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df[col] = df[col].apply(clean_flag)


In [8]:
# --- Safe Getter to Avoid Series Issues ---
def get_scalar(row, col_name):
    try:
        val = row.get(col_name, False)
        if isinstance(val, (pd.Series, list)):
            val = next((v for v in val if pd.notna(v)), False)
        return bool(val)
    except Exception as e:
        print(f"[ERROR] Getting Scalar for {col_name}: {e}")
        return False

# --- Updated Summary Functions ---
def summarize_needs(row):
    needs = []
    if get_scalar(row, 'Wants_Automation'): needs.append('Automation Service')
    if get_scalar(row, 'Wants_Team_Training'): needs.append('Team Training')
    if get_scalar(row, 'Wants_Personal_Course'): needs.append('Personal Course')
    return ', '.join(needs) if needs else 'No AI Need'

def summarize_investments(row):
    levels = []
    if get_scalar(row, 'Invest_PreSeed'): levels.append('Pre-Seed')
    if get_scalar(row, 'Invest_Seed'): levels.append('Seed')
    if get_scalar(row, 'Invest_SeriesA'): levels.append('Series A')
    if get_scalar(row, 'Invest_SeriesB'): levels.append('Series B')
    if get_scalar(row, 'Invest_SeriesC'): levels.append('Series C+')
    return ', '.join(levels) if levels else 'No Investment Interest'

def summarize_regions(row):
    regions = []
    for region_col in region_columns:
        if get_scalar(row, region_col):
            regions.append(region_col.replace('Region_', '').replace('_', ' '))
    return ', '.join(regions) if regions else 'No Regional Focus'

# --- Apply Summary Columns Again ---
cleaned_df['AI_Need_Summary'] = cleaned_df.apply(summarize_needs, axis=1)
cleaned_df['Investment_Interest'] = cleaned_df.apply(summarize_investments, axis=1)
cleaned_df['Region_Focus'] = cleaned_df.apply(summarize_regions, axis=1)

# --- Final Output ---
print(cleaned_df.head())


             Name LinkedIn                                            Company  \
0  🚀 Tyler Kelley      NaN                        SLAM + Outsourced Marketing   
1          A Amin      NaN                                 Operations manager   
2             A D      NaN                                   Edit on the Spot   
3        a data 0      NaN                                               Self   
4           A.D R      NaN  I am transforming as AI consultant my clients ...   

                    Website  Wants_Automation  Wants_Automation  \
0            slamagency.com              True             False   
1                        na              True             False   
2                       NaN              True             False   
3                       NaN              True             False   
4  Will be re-launched soon              True             False   

   Wants_Automation  Wants_Automation  Wants_Automation  Wants_Team_Training  \
0             False           

In [12]:
cleaned_df.isna().sum()

Name                      0
LinkedIn               2920
Company                  75
Website                1725
Wants_Automation          0
                       ... 
Region_UK                 0
Region_UK                 0
AI_Need_Summary           0
Investment_Interest       0
Region_Focus              0
Length: 76, dtype: int64

In [14]:
# --- Columns to Fill ---
columns_to_fill = ['Name', 'LinkedIn', 'Company', 'Website']

# --- Ensure columns_to_fill exist in df (handles duplicates by selecting all matching columns) ---
existing_cols = [col for col in cleaned_df.columns if any(col.startswith(target) for target in columns_to_fill)]

# --- Replace NaNs in these columns with 'Not Listed' ---
cleaned_df[existing_cols] = cleaned_df[existing_cols].fillna('Not Listed')

# --- Verify ---
print(cleaned_df[existing_cols].head())


             Name    LinkedIn  \
0  🚀 Tyler Kelley  Not Listed   
1          A Amin  Not Listed   
2             A D  Not Listed   
3        a data 0  Not Listed   
4           A.D R  Not Listed   

                                             Company                   Website  
0                        SLAM + Outsourced Marketing            slamagency.com  
1                                 Operations manager                        na  
2                                   Edit on the Spot                Not Listed  
3                                               Self                Not Listed  
4  I am transforming as AI consultant my clients ...  Will be re-launched soon  


In [19]:
# Deduplicate Column Names by appending .1, .2 if duplicated
def deduplicate_columns(columns):
    counts = {}
    new_columns = []
    for col in columns:
        if col in counts:
            counts[col] += 1
            new_columns.append(f"{col}.{counts[col]}")
        else:
            counts[col] = 0
            new_columns.append(col)
    return new_columns

# Apply deduplication to cleaned_df columns
cleaned_df.columns = deduplicate_columns(cleaned_df.columns)


In [20]:
from pandas import ExcelWriter
# --- Export All DataFrames to One Excel File with Multiple Sheets ---
output_path = 'Cleaned_Data/AI_Showcase_Cleaned_Export.xlsx'

# Prepare styled DataFrame with colors for AI_Need_Summary
def highlight_needs(val):
    if 'Automation Service' in val:
        return 'background-color: #B6E2D3'
    elif 'Team Training' in val:
        return 'background-color: #FFD699'
    elif 'Personal Course' in val:
        return 'background-color: #FF9999'
    return ''

styled_cleaned_df = cleaned_df.style.applymap(highlight_needs, subset=['AI_Need_Summary'])

# --- Export using ExcelWriter ---
with ExcelWriter(output_path, engine='openpyxl') as writer:
    # Sheet 1: Master Combined List (Raw)
    combined_df.to_excel(writer, sheet_name='Master_Combined_List', index=False)

    # Sheet 2: Cleaned & Summarized Data
    cleaned_df.to_excel(writer, sheet_name='Cleaned_AI_Needs', index=False)

    # Sheet 3: Styled AI Needs Summary
    styled_cleaned_df.to_excel(writer, sheet_name='Styled_AI_Needs', index=False)

print(f"[Exported] {output_path} with 3 Sheets — (Master_Combined_List, Cleaned_AI_Needs, Styled_AI_Needs)")


C:\Users\user\AppData\Local\Temp\ipykernel_2440\1656916536.py:15: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_cleaned_df = cleaned_df.style.applymap(highlight_needs, subset=['AI_Need_Summary'])


[Exported] Cleaned_Data/AI_Showcase_Cleaned_Export.xlsx with 3 Sheets — (Master_Combined_List, Cleaned_AI_Needs, Styled_AI_Needs)
